# Day 08

## Process input data

In [5]:
def read_data(filename):
    with open(filename) as fh:
        data = []
        for row in fh.readlines():
            row = row.strip()
            operation, counter = row.split(" ")
            counter = int(counter)
            data.append((operation, counter))
    return data

In [11]:
data = read_data("input.txt")
data[:10]

[('acc', 14),
 ('acc', 11),
 ('nop', 422),
 ('acc', 14),
 ('jmp', 443),
 ('acc', 21),
 ('nop', 524),
 ('acc', -2),
 ('jmp', 279),
 ('jmp', 1)]

## Part 1

In [12]:
executed = []
accumulator = 0
pointer = 0

In [13]:
while pointer not in executed:
    operation, counter = data[pointer]
    executed.append(pointer)
    
    if operation == "acc":
        accumulator += counter
        pointer += 1
    if operation == "jmp":
        pointer += counter
    if operation == "nop":
        pointer += 1
        
accumulator

1675

## Read data for GPU

In [1]:
import numpy as np
from numba import cuda

In [2]:
intruction_key = {
    "acc": 0,
    "jmp": 1, 
    "nop": 2
}

def read_data_gpu(filename):
    with open(filename) as fh:
        data = []
        for row in fh.readlines():
            row = row.strip()
            operation, counter = row.split(" ")
            counter = int(counter)
            operation = intruction_key[operation]
            data.append(np.asarray([operation, counter]))
    return np.asarray(data)

In [30]:
instructions = read_data_gpu("input.txt")
instructions

array([[  0,  14],
       [  0,  11],
       [  2, 422],
       ...,
       [  0,  22],
       [  0,  10],
       [  1,   1]])

In [31]:
[bits_to_flip] = np.where(instructions[:, 0]>0)
bits_to_flip

array([  2,   4,   6,   8,   9,  12,  16,  17,  19,  21,  22,  25,  27,
        29,  31,  33,  38,  39,  43,  47,  48,  50,  51,  52,  55,  57,
        58,  59,  63,  64,  69,  74,  77,  79,  83,  86,  87,  91,  94,
        99, 100, 101, 103, 106, 107, 111, 114, 115, 116, 120, 121, 122,
       125, 130, 132, 135, 136, 140, 143, 144, 148, 149, 150, 151, 156,
       159, 160, 161, 166, 171, 175, 176, 181, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 200, 203, 206, 207, 209, 210, 213, 214,
       218, 219, 220, 221, 222, 223, 224, 226, 229, 232, 237, 239, 241,
       243, 244, 246, 247, 250, 252, 254, 256, 257, 261, 265, 269, 271,
       273, 274, 275, 277, 278, 280, 281, 284, 286, 287, 289, 291, 292,
       294, 297, 298, 300, 301, 302, 303, 304, 306, 310, 311, 315, 316,
       317, 320, 322, 325, 326, 327, 329, 330, 334, 336, 338, 343, 346,
       348, 351, 353, 356, 357, 358, 359, 360, 363, 364, 366, 368, 370,
       372, 373, 376, 377, 381, 382, 383, 384, 388, 393, 394, 39

In [32]:
@cuda.jit
def brute_force_flips(bits_to_flip, instructions, output):
    pos = cuda.grid(1)
    if pos < len(bits_to_flip):
        flip = bits_to_flip[pos]
        
        accumulator = 0
        pointer = 0
        call_count = 0
        
        while call_count <= len(instructions):
            call_count += 1 
            operation, counter = instructions[pointer]
            
            # Flip bit
            if pointer == flip:
                if operation == 1:
                    operation = 2
                elif operation == 2:
                    operation = 1

            if operation == 0:
                accumulator += counter
                pointer += 1
            if operation == 1:
                pointer += counter
            if operation == 2:
                pointer += 1
                
            if pointer == len(instructions):
                output[pos] = accumulator
                break

In [33]:
threadsperblock = 128
blockspergrid = (len(bits_to_flip) + (threadsperblock - 1)) // threadsperblock

In [34]:
output = np.zeros(len(bits_to_flip))
brute_force_flips[blockspergrid, threadsperblock](bits_to_flip, instructions, output)
max(output)

1532.0